In [38]:
import cv2
import numpy as np
from YOLO import Yolo
import time

In [39]:
CLASSES1 = [ "Bird", "Cat", "Dog", "Flower", "Face" ]  # class names for type 1
CLASSES2 = ["Insect", "Fish", "Fast_food", "Animal", "Fruit", "Traffic_light",\
             "Vehicle_registration_plate", "Car", "Weapon"] # class names for type 2
type = 1
classes = CLASSES1 if type == 1 else CLASSES2
SOURCE = 0
classes

['Bird', 'Cat', 'Dog', 'Flower', 'Face']

In [40]:
def transform(frame, img_size = 320):
    im = cv2.resize(frame, (img_size, img_size), 3).astype("float32")
    im = np.expand_dims(im, axis=0)/255.0
    return im

In [41]:
MODEL_PATH =  f'tflite_models/custom0{type}.tflite'
YOLO = Yolo(model_path = MODEL_PATH, CLASSES = classes)


In [42]:
def return_detections(im, H, W):
    YOLO.pred(im)
    YOLO.extract_results()
    YOLO.return_bbox()
    data = YOLO.return_results(H, W)
    return data["final"]

In [43]:
def test_camera(source):
    cap = cv2.VideoCapture(source)
    _, frame = cap.read() 
    print(frame.shape)
    if cap is None or not cap.isOpened():
       print('Unable to open video source: ', source)
    cap.release()
test_camera(SOURCE) # inbuild webcam

(480, 640, 3)


In [44]:
def start():
    capture = cv2.VideoCapture(SOURCE)
    capture.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
    while True:
        k = cv2.waitKey(1) & 0xFF
        stime = time.time()
        ret, frame = capture.read()
        if ret:
            new_image = transform(frame)
            results = return_detections(new_image, frame.shape[0], frame.shape[1])
            print("results are, - ", results)
            for [coordinates, score, class_name] in results:
                coordinates = [int(x) for x in coordinates]

                xmin, ymin, xmax, ymax = coordinates

                color = tuple(255 * np.random.rand(3))
                text = '{}: {:.0f}%'.format(class_name, score)

                frame = cv2.rectangle(frame, (int(xmin),int(ymin)), \
                                    (int(xmax),int(ymax)), color, 3)
                frame = cv2.putText(frame, text, (int(xmin), int(ymin)-10),\
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            cv2.imshow('frame', frame)

            print('FPS {:.1f}'.format(1 / (time.time() - stime)))

        if cv2.waitKey(1) & 0xFF == ord('q') or k%256 == 27:
            print("exiting...")
            capture.release()
            cv2.destroyAllWindows()
            break
start()